## Mobile Telecommunications Industry Equilibrium Simulation Example

Import packages. (Note that this is a subset of the packages imported in $\texttt{counterfactuals.py}$ because we are only importing the ones we need to use.)

In [1]:
import numpy as np
import pandas as pd

import copy

import supply.infrastructurefunctions as infr
import supply.infrastructureequilibrium as ie

import demand.demandsystem as demsys
import demand.coefficients as coef

import welfare.welfare as welfare

import time

Define function to compute equilibrium and measure welfare. Below is a simplified version of the one in  $\texttt{counterfactuals.py}$.

In [2]:
def compute_eqm(bws, gammas, ds, xis, theta, populations, market_areas, c_u, c_R, R_0, p_0, num_firms=None, num_prods=None, product_firm_correspondence=None, areatype="urban", impute_MVNO={'impute': False}, symmetric=False, R_fixed=False, print_addl_msg=False, pidx=0, qidx=1):
    """
        Compute an equilibrium given description of market structure
    
    Parameters
    ----------
        bws : ndarray
            (M,F) or (M,1) array of bandwidth in MHz
        gammas : ndarray
            (M,) array of spectral efficiencies
        ds : DemandSystem
            contains all the data about markets for demand
        xis : ndarray 
            (M,J_total) or (M,J_{per firm}) matrix of xis
        theta : ndarray
            (K,) array of demand parameters
        populations : ndarray
            (M,) array of market populations
        market_areas : ndarray
            (M,) array of geographic size of markets in km^2
        c_u : ndarray
            (J_total,) or (J_{per firm},) array of per-user costs
        c_R : ndarray
            (M,F) or (M,1) array of per-tower costs
        R_0 : ndarray
            (M,F) or (M,1) array of initial guess of firm-market radii
        p_0 : ndarray
            (J_total,) or (J_{per firm},) array of initial guess of prices
        num_firms : int or None
            specifies the number of firms if symmetric
        num_prods : int or None
            specifies the number of products per firm if symmetric
        product_firm_correspondence : ndarray or None
            (J_total,) array of integers that specify id of firm to which product belongs if not symmetric
        areatype : string
            whether using urban, suburban, or rural Hata loss function
        symmetric : bool
            specifies whether the equilibrium solving for is symmetric (quicker to compute
        impute_MVNO : dict
            dict with
            'impute' : bool (whether to impute the Qs for MVNO)
            'firms_share' (optional) : ndarray ((F-1,) array of whether firms share qualities with MVNOs)
            'include' (optional) : bool (whether to include MVNO Q in returned Q)
        R_fixed : bool
            determine whether to allow R to respond in the equilibrium, if True use R_0 as R*
        print_addl_msg : bool
            determines whether or not to print inputs and output of root solver
        pidx : int
            index of prices in ds
        qidx : int
            index of download speeds in ds
            
    Returns
    -------
        successful : bool
            True if equilibrium computation successful, False otherwise
        cs_ : float
            consumer surplus
        ps_ : float
            producer surplus
        ts_ : float
            total surplus
        p_star_ : ndarray
            (J_total,) or (J_{per firm},) array of firms' equilibrium prices
        R_star_ : ndarray
            (M,F) or (M,) array of firms' equilibrium infrastrucuture choices
        num_stations_stars_ : ndarray
            (M,) array of total number of base stations built
        num_stations_per_firm_stars_ : ndarray
            (M,F) or (M,) array of total number of base stations built by each firm
        q_star_ : ndarray
            (M,F) or (M,) array of download speeds that result from prices and infrastructure
    """

    # Deep copy DemandSystem so can make changes to it and not change original
    ds_ = copy.deepcopy(ds)

    # Compute the equilibrium
    R_star, p_star, q_star, success = ie.infrastructure_eqm(bws, gammas, ds_, xis, theta, populations, market_areas, c_u, c_R, R_0, p_0, symmetric=symmetric, impute_MVNO=impute_MVNO, areatype=areatype, R_fixed=R_fixed, print_msg=print_addl_msg)

    # Update Demand System
    if symmetric:
        ds_.data[:,:,pidx] = np.copy(p_star)
        ds_.data[:,:,qidx] = np.tile(q_star, (num_prods,))
    else:
        ds_.data[:,:,pidx] = p_star[np.newaxis,:]
        ds_.data[:,:,qidx] = np.take_along_axis(q_star, product_firm_correspondence[np.newaxis,:], 1)

    # Calculate welfare impact
    if symmetric:
        xis = np.tile(xis, (1,num_firms))
        c_u = np.tile(c_u, (num_firms,))
        c_R = np.tile(c_R, (1,num_firms))
    cs_ = welfare.agg_consumer_surplus(ds_, xis, theta, populations, include_logit_shock=False, include_pop=False)
    ps_ = welfare.producer_surplus(ds_, xis, theta, populations, market_areas, R_star, c_u, c_R, include_pop=False)
    ts_ = cs_ + ps_

    # Determine equilibrium values to return
    p_stars_ = p_star[:num_prods] if symmetric else np.copy(p_star)
    R_stars_ = R_star[:,0] if symmetric else R_star
    num_stations_stars_ = num_firms * infr.num_stations(R_stars_, market_areas) if symmetric else np.sum(infr.num_stations(R_stars_, market_areas), axis=1)
    num_stations_per_firm_stars_ = infr.num_stations(R_stars_, market_areas)
    q_stars_ = q_star[:,0] if symmetric else q_star
            
    return success, cs_, ps_, ts_, p_stars_, R_stars_, num_stations_stars_, num_stations_per_firm_stars_, q_stars_

Set up the environment with the same values used as when we solve for the symmetric equilibrium with four firms. Because we are solving for a symmetric equilibrium (with $\texttt{symmetric}$ option in $\texttt{compute}{\_}\texttt{eqm}$ set to TRUE), we do not need to repeat the firm dimension for each firm.

In [3]:
# Number of firms and bandwidth endowments
num_firms = 4
bws = np.array([[77.8258306075]]) # per-firm bandwidth allocations
num_markets = bws.shape[0]

# Per-base station costs
c_R = np.array([[3333.43898256]]) # cost per-base station (this is per-bw cost multiplied by the amount of bandwidth)

# Description of market(s)
gammas = np.array([0.1615156]) # spectral efficiencies
populations = np.array([45502.2951795]) # population of markets
market_areas = np.array([16.299135]) # area of markets
income_distributions = np.array([[4308.1, 6636.6, 8778.3, 10723.2, 12722., 14742.4, 17051.2, 20040., 24792.1]]) # income deciles

# Phone plans offered
num_prods = 2 # per firm
dlims = np.array([1000.0, 10000.0]) # data limits of each plan offered by each firm
vlims = np.array([1.0, 1.0]) # unlimited voice dummies of each plan offered by each firm
c_u = np.array([8.1754159, 20.53066142]) # marginal costs

# Demand
theta = np.array([-1.8593453, -0.72733838, 0.46040311, 2.37549113, 0.59651453, 0.33457959, -8.87018317, 0.76662816]) # vector of demand parameters
xis = np.array([[2.37549113, 2.37549113]]) # \xi's of each plan in each market

Create a DemandSystem with the above information. If not changing product characteristics, there is no need to edit the below code.

In [4]:
# Set up initial DemandSystem
chars = {'names': ['p', 'q', 'dlim', 'vunlimited', 'Orange'], 'norm': np.array([False, False, False, False, False])} # product characteristics
elist = [] # spectral efficiencies, not used, so can leave blank
demolist = ['yc1', 'yc2', 'yc3', 'yc4', 'yc5', 'yc6', 'yc7', 'yc8', 'yc9'] # income deciles names
col_names = chars['names'] + ['dbar', 'pop_dens'] + demolist + ['mktshare', 'msize', 'market', 'j'] # some of these required but only used for demand estimation, so values don't matter for simulation
df_ds = pd.DataFrame({col: np.arange(num_prods) + 1 for col in col_names}) # values don't matter, will be replaced later
ds = demsys.DemandSystem(df_ds, chars, elist, demolist, np.zeros((1,)), 0.0, np.arange(num_prods), np.arange(num_prods), 0.0)

# Identify indices to change plan characteristics
pidx = ds.chars.index(ds.pname)
qidx = ds.chars.index(ds.qname)
dlimidx = ds.chars.index(ds.dlimname)
vlimidx = ds.chars.index(ds.vunlimitedname)
Oidx = ds.chars.index(ds.Oname)
yc1idx = ds.dim3.index(ds.demolist[0])
yclastidx = ds.dim3.index(ds.demolist[-1])

# Fill in plan and market characteristics
ds.data = np.zeros((num_markets, num_firms * num_prods,ds.data.shape[2])) 
ds.data[:,:,pidx] = np.zeros((num_markets, num_firms * num_prods)) # prices, doesn't matter b/c will be replaced in compute_eqm function
ds.data[:,:,qidx] = np.zeros((num_markets, num_firms * num_prods)) # download speeds, doesn't matter b/c will be replaced in compute_eqm function
ds.data[:,:,dlimidx] = np.tile(dlims[np.newaxis,:], (num_markets, num_firms)) # data limits
ds.data[:,:,vlimidx] = np.tile(vlims[np.newaxis,:], (num_markets, num_firms)) # voice limits
ds.data[:,:,Oidx] = 0. # Orange dummy, zero b/c captured by the \xi's
ds.firms = np.repeat(np.arange(num_firms, dtype=int) + 1, num_prods) # id for each firm
ds.J = num_firms * num_prods # total number of phone plans
ds.data[:,:,yc1idx:yclastidx+1] = income_distributions[:,np.newaxis,:] # fill in income distribution

Now we can compute the equilibrium.

In [5]:
# Starting values for radii and prices
R_0 = np.array([[0.5]]) # initial guess of equilibrium radii
p_0 = np.array([8.1754159, 20.53066142]) # initial guess of equilibrium prices

start = time.time()
res = compute_eqm(bws, gammas, ds, xis, theta, populations, market_areas, c_u, c_R, R_0, p_0, num_firms=num_firms, num_prods=num_prods, symmetric=True)
end = time.time()
success = res[0]
cs_ = res[1]
ps_ = res[2]
ts_ = res[3]
p_stars_ = res[4]
R_stars_ = res[5]
num_stations_stars_ = res[6]
num_stations_per_firm_stars_ = res[7]
q_stars_ = res[8]

if success:
    print(f"Successful computation in {np.round(end - start, 1)} seconds. Equilibrium values:", flush=True)
    print(f"\tprices: {p_stars_}", flush=True)
    print(f"\tradii: {R_stars_}", flush=True)
    print(f"\tnumber of total stations: {num_stations_stars_}", flush=True)
    print(f"\tnumber of stations per firm: {num_stations_per_firm_stars_}", flush=True)
    print(f"\tdownload speeds: {q_stars_}", flush=True)
    print(f"\tconsumer surplus: {cs_}", flush=True)
    print(f"\tproduct surplus: {ps_}", flush=True)
    print(f"\ttotal surplus: {ts_}", flush=True)
else:
    print(f"Equilibrium computation failed.", flush=True)

Successful computation in 19.4 seconds. Equilibrium values:
	prices: [14.67693195 29.86382408]
	radii: [1.73331385]
	number of total stations: [8.35253386]
	number of stations per firm: [2.08813346]
	download speeds: [2.90203298]
	consumer surplus: 35.6166842090322
	product surplus: 5.519399060128836
	total surplus: 41.136083269161034


Instead of using the $\texttt{symmetric}$ option, we could have repeated the values for each of the four firms and solved for the equilibrium, as follows.

In [6]:
# Expand variables across the firm dimension
bws = np.array([[77.8258306075, 77.8258306075, 77.8258306075, 77.8258306075]])
c_R = np.array([[3333.43898256, 3333.43898256, 3333.43898256, 3333.43898256]])
c_u = np.array([8.1754159, 20.53066142, 8.1754159, 20.53066142, 8.1754159, 20.53066142, 8.1754159, 20.53066142])
xis = np.array([[2.37549113, 2.37549113, 2.37549113, 2.37549113, 2.37549113, 2.37549113, 2.37549113, 2.37549113]])
R_0 = np.array([[0.5, 0.5, 0.5, 0.5]])
p_0 = np.array([8.1754159, 20.53066142, 8.1754159, 20.53066142, 8.1754159, 20.53066142, 8.1754159, 20.53066142])

# Create array mapping for each product which firm it belongs to
product_firm_correspondence = np.array([0, 0, 1, 1, 2, 2, 3, 3])

# Solve for equilibrium
start = time.time()
res = compute_eqm(bws, gammas, ds, xis, theta, populations, market_areas, c_u, c_R, R_0, p_0, product_firm_correspondence=product_firm_correspondence)
end = time.time()
success = res[0]
cs_ = res[1]
ps_ = res[2]
ts_ = res[3]
p_stars_ = res[4]
R_stars_ = res[5]
num_stations_stars_ = res[6]
num_stations_per_firm_stars_ = res[7]
q_stars_ = res[8]

if success:
    print(f"Successful computation in {np.round(end - start, 1)} seconds. Equilibrium values:", flush=True)
    print(f"\tprices: {p_stars_}", flush=True)
    print(f"\tradii: {R_stars_}", flush=True)
    print(f"\tnumber of total stations: {num_stations_stars_}", flush=True)
    print(f"\tnumber of stations per firm: {num_stations_per_firm_stars_}", flush=True)
    print(f"\tdownload speeds: {q_stars_}", flush=True)
    print(f"\tconsumer surplus: {cs_}", flush=True)
    print(f"\tproduct surplus: {ps_}", flush=True)
    print(f"\ttotal surplus: {ts_}", flush=True)
else:
    print(f"Equilibrium computation failed.", flush=True)

Successful computation in 154.8 seconds. Equilibrium values:
	prices: [14.67693195 29.86382409 14.67693195 29.86382407 14.67693195 29.8638241
 14.67693195 29.86382407]
	radii: [[1.73331385 1.73331385 1.73331385 1.73331385]]
	number of total stations: [8.35253387]
	number of stations per firm: [[2.08813347 2.08813347 2.08813347 2.08813347]]
	download speeds: [[2.90203299 2.90203299 2.90203299 2.90203299]]
	consumer surplus: 35.61668420805435
	product surplus: 5.519399061005499
	total surplus: 41.13608326905985


We can even solve for asymmetric equilibria. For example, let's change the bandwidth allocations of the firms.

In [7]:
# Change bandwidth allocation
per_bw_c_R = c_R / bws
bws = np.array([[0.2, 0.2, 0.3, 0.3]]) * np.sum(bws) # preserve the total amount of bandwidth
c_R = per_bw_c_R * bws # changing bandwidth changes per-base station cost

# Solve for equilibrium
start = time.time()
res = compute_eqm(bws, gammas, ds, xis, theta, populations, market_areas, c_u, c_R, R_0, p_0, product_firm_correspondence=product_firm_correspondence)
end = time.time()
success = res[0]
cs_ = res[1]
ps_ = res[2]
ts_ = res[3]
p_stars_ = res[4]
R_stars_ = res[5]
num_stations_stars_ = res[6]
num_stations_per_firm_stars_ = res[7]
q_stars_ = res[8]

if success:
    print(f"Successful computation in {np.round(end - start, 1)} seconds. Equilibrium values:", flush=True)
    print(f"\tprices: {p_stars_}", flush=True)
    print(f"\tradii: {R_stars_}", flush=True)
    print(f"\tnumber of total stations: {num_stations_stars_}", flush=True)
    print(f"\tnumber of stations per firm: {num_stations_per_firm_stars_}", flush=True)
    print(f"\tdownload speeds: {q_stars_}", flush=True)
    print(f"\tconsumer surplus: {cs_}", flush=True)
    print(f"\tproduct surplus: {ps_}", flush=True)
    print(f"\ttotal surplus: {ts_}", flush=True)
else:
    print(f"Equilibrium computation failed.", flush=True)

Successful computation in 232.3 seconds. Equilibrium values:
	prices: [14.66312494 29.85528926 14.66312494 29.85528927 14.69002039 29.87160725
 14.6900204  29.87160731]
	radii: [[1.60322889 1.60322889 1.85049547 1.85049547]]
	number of total stations: [8.54557511]
	number of stations per firm: [[2.44074052 2.44074052 1.83204703 1.83204703]]
	download speeds: [[2.78333828 2.78333828 2.99570433 2.99570432]]
	consumer surplus: 35.61114773300332
	product surplus: 5.5226027050154105
	total surplus: 41.13375043801873


If we additionally wanted to change the data limits of the phone plans by firm, we could do that too. For example, let's have 8 GB instead of 10 GB as the upper data limit for some firms (one of the 0.2 bw firms and one of the 0.3 bw firms).

In [8]:
# Replace data limit characteristics
new_dlims = np.array([1000.0, 10000.0, 1000.0, 8000.0, 1000.0, 10000.0, 1000.0, 8000.0])
ds.data[:,:,dlimidx] = np.tile(new_dlims[np.newaxis,:], (num_markets, 1))

# Solve for equilibrium
start = time.time()
res = compute_eqm(bws, gammas, ds, xis, theta, populations, market_areas, c_u, c_R, R_0, p_0, product_firm_correspondence=product_firm_correspondence)
end = time.time()
success = res[0]
cs_ = res[1]
ps_ = res[2]
ts_ = res[3]
p_stars_ = res[4]
R_stars_ = res[5]
num_stations_stars_ = res[6]
num_stations_per_firm_stars_ = res[7]
q_stars_ = res[8]

if success:
    print(f"Successful computation in {np.round(end - start, 1)} seconds. Equilibrium values:", flush=True)
    print(f"\tprices: {p_stars_}", flush=True)
    print(f"\tradii: {R_stars_}", flush=True)
    print(f"\tnumber of total stations: {num_stations_stars_}", flush=True)
    print(f"\tnumber of stations per firm: {num_stations_per_firm_stars_}", flush=True)
    print(f"\tdownload speeds: {q_stars_}", flush=True)
    print(f"\tconsumer surplus: {cs_}", flush=True)
    print(f"\tproduct surplus: {ps_}", flush=True)
    print(f"\ttotal surplus: {ts_}", flush=True)
else:
    print(f"Equilibrium computation failed.", flush=True)

Successful computation in 148.7 seconds. Equilibrium values:
	prices: [14.66304968 29.85524573 14.6631573  29.85530327 14.6899426  29.87156179
 14.69015408 29.87167967]
	radii: [[1.60330241 1.60281023 1.85057677 1.84960235]]
	number of total stations: [8.54823523]
	number of stations per firm: [[2.44051668 2.44201574 1.83188607 1.83381673]]
	download speeds: [[2.78321535 2.78413286 2.99556931 2.99737845]]
	consumer surplus: 35.61139871089839
	product surplus: 5.52241378580884
	total surplus: 41.13381249670723
